In [ ]:
import logging, json

import numpy as np

from pypower.ext2int import ext2int
from alive_progress import alive_bar

import covee.control_strategies.utils as utils
from covee.csv_files.save_results import save_results
from covee.powerflow.runPF_class import runPF_class


logging.basicConfig(level = logging.DEBUG)
logging.debug('Just started')

with open('examples/conf.json') as f:
    conf_dict = json.load(f)                                                                                                    # conf_dict is a dictionary contains the imported json data

del(f)

module_obj = utils.select(conf_dict)                                                                                            # module_obj is a covee.control_strategies.utils.select object with its conf_dict attribute initialized to the json data dictionary

Quadratic_Control = module_obj.select_control()                                                                                 # The control_module attribute of module_obj is initialized to the string 'covee.control_strategies.Quadratic_Control_Centralized_CVXPY'
                                                                                                                                # Quadratic_Control is the python module covee.control_strategies.Quadratic_Control_Centralized_CVXPY

case = module_obj.select_case()                                                                                                 # The case_module attribute of module_obj is initialized to the string 'covee.cases.case30'
                                                                                                                                # case is the python module covee.cases.case30

del(module_obj)

additional = utils.additional()                                                                                                 # additional is an empty covee.control_strategies.utils.additional object

conf_dict = additional.convert_index(conf_dict)                                                                                 # conf_dict attribute of additional is set to the actual conf_dict (with active nodes and active ESS indices starting from 0)
                                                                                                                                # active nodes and active ESS indices of conf_dict start from zero

active_nodes = conf_dict["CONTROL_DATA"]["active_nodes"]
active_ESS = conf_dict["CONTROL_DATA"]["active_ESS"]

ppc = ext2int(case.case_().case())                                                                                              # ppc is a dictionary containing all the case (grid) information; convert bus numbers to start from zero. A covee.cases.case30.case_ object is used to create this

[grid_data, reactive_power, active_power, active_power_ESS] = additional.system_info(ppc, 1, active_nodes, active_ESS)          # the active_nodes and active_ESS attributes of additional are initialized

del(reactive_power)
del(active_power)
del(active_power_ESS)

control = Quadratic_Control.Quadratic_Control(grid_data, active_nodes, active_ESS, conf_dict["CONTROL_DATA"])                   # control is a covee.control_strategies.Quadratic_Control_Centralized_CVXPY.Quadratic_Control object with the attributes grid_data, num_pv, num_ESS, num_bus, control_data, QMIN, QMAX, PMIN, PMAX, V_MIN, V_MAX and V_NOM

del(Quadratic_Control)

[R, X, output] = control.initialize_control()                                                                                   # control object is initialized with the additional attributes bus_values, additional, reactive_power, active_power_PV, active_power_ESS, active_nodes, active_nodes_ESS, active_tot, factor_DG_array, factor_ESS_array, q_DG, p_DG, p_ESS, v, W_Q_DG, W_P_DG, W_P_ESS, W_V, rho_max, and rho_min
                                                                                                                                # R, X, and output are returned by the initialization function


save_obj = save_results(voltage_list = [], iterations = [], conf_dict = conf_dict)                                              # save_obj is a covee.csv_files.save_results.save_results object

run_PF = runPF_class(active_nodes, active_ESS, grid_data["full_nodes"], grid_data["total_control_nodes"])                       # run_PF is a covee.powerflow.runPF_class.runPF_class object

profiles = run_PF.read_profiles(conf_dict, grid_data)

active_power_dict = {}
reactive_power_dict = {}
active_power_ESS_dict = {}

with alive_bar(len(profiles["gen_profile"])) as bar:
    for iter in range(len(profiles["gen_profile"])):
        [v_tot, v_gen, p, c, p_load, v_pv, v_ess] = run_PF.run_Power_Flow(ppc,
                                                                          output["DG"]["active_power"],
                                                                          output["DG"]["reactive_power"],
                                                                          output["ESS"]["active_power"],
                                                                          profiles["gen_profile"][iter],
                                                                          profiles["load_profile"][iter])
        
        print(v_gen)
        del(v_tot)
        del(p)
        del(c)
        del(p_load)
        del(v_pv)
        
        output = control.control_(profiles["gen_profile"][iter][active_nodes],
                                  output,
                                  R,
                                  X,
                                  v_gen,
                                  v_ess,
                                  VMIN = conf_dict["CONTROL_DATA"]["VMIN"],
                                  VMAX = conf_dict["CONTROL_DATA"]["VMAX"],
                                  iter = iter)

        bar()